In [1]:
import pickle
import numpy as np
import tkinter as tk
from tkinter import *
from helpers import *
from tkinter import filedialog as fd
from tkinter import messagebox as mb

def loadConfig(fname):
    config = loadPickle(fname)
    for key in config.keys():
        globals()[str(key)] = config[key]
    globals()['configFile'] = fname
    
try:
    loadConfig('config.pkl')
except:
    print("Failed to load the config file!")
    quit()

In [2]:
class layerGenerator:
    def __init__(self):
        self.h = None
        self.w = None
        self.size = None
        
        self.locs = None
        self.coeffs = None
        self.configName = None
        
        self.quantization_bits = None
        
    def loadLocations(self, enc='latin1'):
        root = Tk()
        root.attributes("-topmost", True)
        root.withdraw()
        i = 0
        while self.locs is None:
            i+= 1
            try:
                fname = fd.askopenfilename(title = "Select locations data", filetypes = (("Pickle Object","*.pkl"),("all files","*.*")))
                with open(fname, "rb" ) as f:
                    self.locs = pickle.load(f , encoding=enc)
                if self.locs.shape[-1] != 7:
                    mb.showerror("Failed!", "Data has wrong dimensions.\nPlease make sure you have selected the correct file", parent=root)
                    self.locs = None
            except:
                self.locs = None
                mb.showerror("Failed!", "Something went wrong while trying to load location data.\nPlease make sure you have selected the correct file and that the file is not in use by other processes.", parent=root)
            if self.locs is None and i>=3:
                if mb.askyesno("Data not loaded correctly!", "Would you like to abort?", parent=root):
                    break
        if self.locs is None:
            quit()
        root.destroy()
#         w = 2*int(np.max(np.abs(self.locs[:,0])) + np.max(self.locs[:,6]//2)+1)
#         h = 2*int(np.max(np.abs(self.locs[:,1])) + np.max(self.locs[:,6]//2)+1)
        w = 2*int(np.abs(self.locs[:,0]).max() + self.locs[:,6].max()/2.0)
        h = 2*int(np.abs(self.locs[:,1]).max() + self.locs[:,6].max()/2.0)
        self.size = np.array([h,w], dtype='int32')
        
    def loadCoefficients(self, enc='latin1'):
        root = Tk()
        root.attributes("-topmost", True)
        root.withdraw()
        i = 0
        while self.coeffs is None:
            i+= 1
            try:
                fname = fd.askopenfilename(title = "Select coefficients data", filetypes = (("Pickle Object","*.pkl"),("all files","*.*")))
                with open(fname, "rb" ) as f:
                    self.coeffs = np.squeeze(pickle.load(open(fname, "rb" ), encoding=enc))
                if len(self.coeffs.shape) != 1:
                    mb.showerror("Failed!", "Data has wrong dimensions.\nPlease make sure you have selected the correct file", parent=root)
                    self.coeffs = None
            except:
                self.coeffs = None
                mb.showerror("Failed!", "Something went wrong while trying to load coefficients data.\nPlease make sure you have selected the correct file and that the file is not in use by other processes.", parent=root)
            if self.coeffs is None and i>=3:
                if mb.askyesno("Data not loaded correctly!", "Would you like to abort?", parent=root):
                    break
        if self.coeffs is None:
            quit()
        #################################
        #################################
        #################################
        numFields = len(self.locs)+1
#         self.arrayLength = len(self.locs)+1
#         self.sampledVector = np.zeros(len(self.locs)+1).astype(numpy_types[types['RESULTS']])

    def packPixels(self, outputFile, toInts=True):
        ######################################
        ########### INITIALIZATION ###########
        ######################################
        h, w = self.size
        self.locs[:,:2] = (self.locs[:,:2]+ np.array((w//2,h//2)))
        overlapCounter = np.zeros((h,w),dtype='uint8')
        #######################################
        ###### FINDING OVERLAPPING AREAS ######
        #######################################
        for idx in range(len(self.locs)):
#             size = int(self.locs[idx][6])
#             offset = size/2.0
#             x , y = np.round(self.locs[idx][:2])
#             x1,y1,x2,y2 = int(x-offset),int(y-offset),int(x+offset),int(y+offset)
            size = self.locs[idx][6]
            y1 = int(self.locs[:,1][idx] - size/2+0.5)
            y2 = int(self.locs[:,1][idx] + size/2+0.5)
            x1 = int(self.locs[:,0][idx] - size/2+0.5)
            x2 = int(self.locs[:,0][idx] + size/2+0.5)
            overlapCounter[y1:y2,x1:x2] += 1
        nLayers = np.max(overlapCounter)
        self.coeff_layers = []
        [self.coeff_layers.append(np.zeros((h,w))) for i in range(nLayers)]
        self.index_layers = []
        [self.index_layers.append(np.zeros((h,w))) for i in range(nLayers)]
        ######################################
        ########### PACKING PIXELS ###########
        ######################################
        for idx in range(len(self.locs)):
            values = self.coeffs[idx]
            size = int(self.locs[idx][6])
            offset = size/2.0
            x , y = np.round(self.locs[idx][:2])
            pixelCoords = [(int(x+i-offset), int(y+j-offset), values[j,i]) for j in range(size) for i in range(size)]
            for coord in pixelCoords:
                a , b , c = coord
                for i in range(nLayers):
                    if self.coeff_layers[i][b,a] == 0:
                        self.coeff_layers[i][b,a] = c
                        self.index_layers[i][b,a] = idx+1
                        break
        #######################################
        ###### CONVERTING FLOATS TO INTS ######
        #######################################
        if toInts:
            scaling_factor = ((2**self.quantization_bits)-1) / np.max(self.coeff_layers)
            for i in range(len(self.coeff_layers)):
                self.coeff_layers[i] = (self.coeff_layers[i]*scaling_factor).astype(numpy_types[types['COEFFICIENTS']])
                self.index_layers[i] = self.index_layers[i].astype(numpy_types[types['INDEX']])
        ######################################
        ######### SAVING THE RESULTS #########
        ######################################
        with open(outputFile, "wb") as f:
            pickle.dump((self.coeff_layers,self.index_layers,len(self.locs)+1), f)
            
    def packKernels(self, outputFile, toInts=True):
        ######################################
        ########### INITIALIZATION ###########
        ######################################
        h, w = self.size
        self.locs[:,:2] = (self.locs[:,:2]+ np.array((w//2,h//2)))
        self.coeff_layers = []
        self.index_layers = []
        #######################################
        ########### PACKING KERNELS ###########
        #######################################
        for idx in range(len(self.locs)):
#             size = int(self.locs[idx][6])
#             offset = size/2.0
#             x , y = np.round(self.locs[idx][:2])
#             x1,y1,x2,y2 = int(x-offset),int(y-offset),int(x+offset),int(y+offset)
            size = self.locs[idx][6]
            y1 = int(self.locs[:,1][idx] - size/2+0.5)
            y2 = int(self.locs[:,1][idx] + size/2+0.5)
            x1 = int(self.locs[:,0][idx] - size/2+0.5)
            x2 = int(self.locs[:,0][idx] + size/2+0.5)
            found_empty_layer = False
            for x in range(len(self.coeff_layers)):
                coeff_layer = self.coeff_layers[x]
                index_layer = self.index_layers[x]
                if len(coeff_layer[np.where(coeff_layer[y1:y2,x1:x2]>0)])==0:
                    coeff_layer[y1:y2,x1:x2] = self.coeffs[idx]
                    index_layer[y1:y2,x1:x2] = idx+1
                    found_empty_layer = True
                    break
            if not found_empty_layer:
                self.coeff_layers.append(np.zeros((h,w)))
                self.index_layers.append(np.zeros((h,w)))

                self.coeff_layers[-1][y1:y2,x1:x2] = self.coeffs[idx]
                self.index_layers[-1][y1:y2,x1:x2] = idx+1
        #######################################
        ###### CONVERTING FLOATS TO INTS ######
        #######################################
        if toInts:
            scaling_factor = ((2**self.quantization_bits)-1) / np.max(self.coeff_layers)
            for i in range(len(self.coeff_layers)):
                self.coeff_layers[i] = (self.coeff_layers[i]*scaling_factor).astype(numpy_types[types['COEFFICIENTS']])
                self.index_layers[i] = self.index_layers[i].astype(numpy_types[types['INDEX']])
        else:
            scaling_factor = 1
        ######################################
        ######### SAVING THE RESULTS #########
        ######################################
        with open("layers.pkl", "wb") as f:
            pickle.dump((self.coeff_layers,self.index_layers,scaling_factor), f)
#         with open(self.configName, "wb") as f:
#             pickle.dump(self.cfg, f)
        config = {'types' : types,
                  'numpy_types' : numpy_types,
                  'numFields' : numFields}
        savePickle(configFile, config)
        
            
##############################################################################################################
##############################################################################################################
##############################################################################################################
##############################################################################################################
##############################################################################################################


    def validate(self):
        self.locs[:,:2] = (self.locs[:,:2]+ np.array((500,500)))
        blnk = np.zeros((2000,2000))
        
#         XX = self.locs[:,0] 
#         YY = self.locs[:,1]
        
        similar = 0
        correct_old = 0
        correct_new = 0
        for idx in range(len(self.locs)):
            size = int(self.locs[idx][6])
            offset = size/2.0
            x , y = np.round(self.locs[idx][:2])
            #x1,y1,x2,y2 = int(x-offset),int(y-offset),int(x+offset),int(y+offset)
            x1,x2 = int(x-offset),int(x+offset)
            y1,y2 = int(y-offset),int(y+offset)
            
            w = self.locs[idx][6]
            yy1 = int(self.locs[:,1][idx] - w/2+0.5)
            yy2 = int(self.locs[:,1][idx] + w/2+0.5)
            xx1 = int(self.locs[:,0][idx] - w/2+0.5)
            xx2 = int(self.locs[:,0][idx] + w/2+0.5)
            
            if (x1==xx1) and (x2==xx2) and (y1==yy1) and (y2==yy2):
                similar += 1
                
            if blnk[y1:y2,x1:x2].shape == (size,size):
                correct_new += 1
                
            if blnk[yy1:yy2,xx1:xx2].shape == (size,size):
                correct_old += 1
                
#             print(size)
#             print('#'*10)    
#             print(x1,x2)
#             print(xx1,xx2)
#             print('#'*10)
#             print(y1,y2)
#             print(yy1,yy2)
#             print('#'*10)
#             print(blnk[y1:y2,x1:x2].shape == (size,size))
#             print(blnk[yy1:yy2,xx1:xx2].shape == (size,size))
#             print('#'*100)

        return (similar,correct_old,correct_new)

    def print_shape(self):
        width = 2*int(np.abs(self.locs[:,:2]).max() + self.locs[:,6].max()/2.0)
        
        w = 2*int(np.max(np.abs(self.locs[:,0])) + np.max(self.locs[:,6]/2))
        h = 2*int(np.max(np.abs(self.locs[:,1])) + np.max(self.locs[:,6]/2))
        
        print(width,width)
        print(h,w)

In [3]:
# G = layerGenerator('config.pkl')
# G.loadLocations()
# G.loadCoefficients()
# G.print_shape()
# print('#'*10)
# print(G.validate())

In [4]:
pack = True
if pack:
    G = layerGenerator()
    G.loadLocations()
    G.loadCoefficients()
    G.quantization_bits = 16
    G.packKernels('new_layers.pkl')